In [1]:
import ee

In [3]:
ee.Initialize()

In [38]:
gebco = ee.Image("users/gena/GEBCO_2014_2D")
emodnet = ee.Image("users/gena/EMODNET")
geom =  ee.Geometry.Polygon([
    [
            [-174.52128906250002, 79.52559147277664],
            [-174.52128906250002, 2.61899508391546],
            [164.38496093749998, 2.61899508391546],
            [164.38496093749998, 79.52559147277664]
        ]
    ], 
    None, 
    False
)
print(geom.getInfo())
land = ee.Image("users/gena/land_polygons_image")

bathy = gebco

{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-174.52128906250002, 2.61899508391546], [164.38496093749998, 2.61899508391546], [164.38496093749998, 79.52559147277664], [-174.52128906250002, 79.52559147277664], [-174.52128906250002, 2.61899508391546]]]}


In [41]:
# Units in m
# Upsample (bathy is rather coarse, so we make scale a bit)
upsample = 10

resolution = 10

tileSize = 256 * resolution * upsample
radius = tileSize / 2

# focal max -> buffer with 5px  -> 250000 / 50 ->  500m -> 2500m buffer.
# use 500m resolution for buffer
landDilated = land.mask().focal_max(5).reproject(**{ "crs": 'EPSG:4326', "scale": radius / 5 })

# lookup region where bathy is undeep  and not above SL and not in buffered land polygon
mask = (
    bathy
    .gt(-50)
    .And(bathy.lt(-1))
    .And(landDilated.Not())
)

# Create a large number of samples
# Filter samples by region
# Like computing pi by throwing darts at a dartboard in a square.... 
# but then less efficient.
samples = (
    bathy
    .updateMask(mask)
    # divide by the factor 
    .divide(upsample)
    .sample(**{ 
        "region": geom.getInfo(), 
        'scale': 1000, 
        "numPixels": 50000, 
        "seed": 42, 
        "dropNulls": True, 
        "geometries": True
    })
)

print(samples.size().getInfo())

# Draw boxes around the buffer (consider sampling by sqrt(2) and  then sampling rotated)
def draw_box(f):
    return f.buffer(radius).bounds()
export_rects = samples.map(draw_box)


# Export tile to bucket
def export_dem(i, bounds):
    task = ee.batch.Export.image.toDrive(**{
        "image": bathy, 
        "description": f'bathymetry-{i}', 
        "folder": 'bathymetry-random-tiles', 
        "fileNamePrefix": f'bathymetry-{i}', 
        "dimensions": '256x256', 
        "region": bounds, 
    })
    return task


export_rects_list = export_rects.toList(export_rects.size())

def export_dem_feature(i):
    task = export_dem(i, ee.Feature(export_rects_list.get(i)).geometry())
    return task

tasks = []
for f in ee.List.sequence(0, export_rects.size()).getInfo():
    task = export_dem_feature(f)
    tasks.append(task)

1125


In [42]:
for task in tasks[:10]:
    task.start()